In [1]:
#Import Packages
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import json_normalize

In [2]:
#Prep for API Call
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': 'INSERT YOUR API KEY HERE'}

### Demo 1: Population in Each Heat Stress Category in Each State

In [ ]:
#Set Up & Make Request
#group_by Endpoint
endpoint = 'zonal_stats/group_by/climate_engine_asset'

#Get population in heat stress categories in each state
#Categories: Extreme Cold Stress, Very Strong Cold Stress, Strong Cold Stress, Moderate Cold Stress, Slight Cold Stress, No Thermal Stress, Moderate Heat Stress, Strong Heat Stress, Very Strong Heat Stress, Extreme Heat Stress
params ={
    'group_img_id': 'projects/climate-engine-pro/assets/ce-era5-heat/20240108',
    'summary_img_id': 'CIESIN/GPWv411/GPW_Population_Count/gpw_v4_population_count_rev11_2020_30_sec',
    'region': 'states',
    'filter_by': 'Name',
    'group_band': 'utci_mean',
    'summary_band': 'population_count',
    'group_img_type': 'continuous',
    'area_reducer': 'sum',
    'bins': '[233.15, 246.15, 286.15, 273.15, 282.15, 299.15, 305.15, 311.15, 319.15]'
  }
# Send request to the API
r = requests.get(root_url + endpoint, params=params, headers=headers, verify=True)
response = r.json()
print(response)

In [ ]:
#Process Output
df = pd.DataFrame(response)

# Flatten the 'group' column
df = pd.concat([df.drop(['group'], axis=1), df['group'].apply(pd.Series)], axis=1)

#rename the columns
df = df.rename(columns = {'(-inf, 233.15)': 'Extreme Cold Stress', '[233.15, 246.15)':'Very Strong Cold Stress', '[246.15, 286.15)': 'Strong Cold Stress','[286.15, 273.15)':'Moderate Cold Stress','[273.15, 282.15)':'Slight Cold Stress','[282.15, 299.15)':'No Thermal Stress','[299.15, 305.15)':'Moderate Heat Stress','[305.15, 311.15)':'Strong Heat Stress','[311.15, 319.15)':'Very Strong Heat Stress','[319.15, inf)':'Extreme Heat Stress'})

#Get Row Total
df['Row Total'] = df.iloc[:, 1:].sum(axis=1)

#Calculate the percent of total population in each stress class
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: (x / df['Row Total'])*100)
df

In [5]:
#Drop Row Total for Visualization
df = df.drop('Row Total', axis=1)

#Sort Alphabetically
df = df.sort_values(by=['Name'],ascending=False)

In [ ]:
#Plot Data
#Create figure
fig, ax = plt.subplots(figsize=(14, 10))

#Plot data as horizontal stacked bar charts
ax = df.plot(kind='barh', stacked = True, color=['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8','#fee090','#fdae61','#f46d43','#d73027','#a50026'], ax=ax)

#Add a title to the plot
plt.title('Jan 08, 2024: % population under different stress categories')

#Label y tick marks using Names
ax.set_yticklabels(df['Name'])

# Add more distance between tick labels
ax.tick_params(axis='y', which='both', pad=10)

#Move legend outside of plot
ax.legend(loc='upper right', bbox_to_anchor=(1.30, 1))

#Make sure the legend isn't cut of when saving figure
plt.tight_layout()

#Save figure to temporary files
plt.savefig('demo1.png')

### Demo 2: Acres of Corn in Drought Categories

In [ ]:
#Set Up & Make Request
#group_by Endpoint
endpoint = 'zonal_stats/group_by/custom_asset'

#Get ha in drought categories in each state
#Categories:
params ={
    'group_img_id': 'GRIDMET/DROUGHT/20220714',
    'summary_img_id': 'projects/climate-engine-pro/assets/cdl-crop-area/2022',
    'asset_id': 'projects/earthengine-legacy/assets/projects/climate-engine/featureCollections/shp_simplified/ClimateEngine_US_States',
    'sub_choices': '[Iowa, Illinois, Nebraska, Minnesota, Indiana]',
    'filter_by': 'Name',
    'group_band': 'spi1y',
    'summary_band': 'crop_area_1',
    'group_img_type': 'continuous',
    'area_reducer': 'sum'
  }
# Send request to the API
r = requests.get(root_url + endpoint, params=params, headers=headers, verify=True)
response = r.json()
print(response)

In [ ]:
#Process Output
df = pd.DataFrame(response)

# Flatten the 'group' column
df = pd.concat([df.drop(['group'], axis=1), df['group'].apply(pd.Series)], axis=1)

#Rename columns
df = df.rename(columns = {'(-inf, -2.5)': 'Exceptional Drought', '[-2.5, -2)':'Extreme Drought', '[-2, -1.5)': 'Severe Drought','[-1.5, -1)':'Moderate Drought','[-1, -0.5)':'Abnormal Dry','[-0.5, 0.5)':'Neutral','[0.5, 1)':'Abnormal Wet','[1, 1.5)':'Moderate Wet','[1.5, 2)':'Severe Wet','[2, 2.5)':'Extreme Wet', '[2.5, inf)': 'Exceptional Wet'})

#Calculate ha
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: (x / 10000))
df

In [ ]:
#Plot data for top 5 producing states
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

#Set bar width
bar_width = 0.3

#Set Starting Points
lengths = np.array([1,4,7,10,13])

# Plot the first set of bars
bars1 = ax.bar(lengths, df['Exceptional Drought'], bar_width, label='Exceptional Drought', color='#a50026')

# Plot the second set of bars, adjusting the x position
bars2 = ax.bar(lengths + bar_width, df['Extreme Drought'], bar_width, label='Extreme Drought', color='#d73027')

# Plot the third set of bars, adjusting the x position
bars3 = ax.bar(lengths + 2*bar_width, df['Severe Drought'], bar_width, label='Severe Drought', color='#f46d43')

# Plot the second set of bars, adjusting the x position
bars4 = ax.bar(lengths + 3*bar_width, df['Moderate Drought'], bar_width, label='Extreme Drought', color='#fdae61')

# Plot the second set of bars, adjusting the x position
bars5 = ax.bar(lengths + 4*bar_width, df['Abnormal Dry'], bar_width, label='Abnormal Dry', color='#fee090')

# Plot the second set of bars, adjusting the x position
bars6 = ax.bar(lengths + 5*bar_width, df['Neutral'], bar_width, label='Neutral', color='#ffffbf')

# Plot the second set of bars, adjusting the x position
bars7 = ax.bar(lengths + 6*bar_width, df['Abnormal Wet'], bar_width, label='Abnormal Wet', color='#e0f3f8')

# Plot the second set of bars, adjusting the x position
bars8 = ax.bar(lengths + 7*bar_width, df['Moderate Wet'], bar_width, label='Moderate Wet', color='#abd9e9')

# Plot the second set of bars, adjusting the x position
bars9 = ax.bar(lengths + 8*bar_width, df['Severe Wet'], bar_width, label='Severe Wet', color='#74add1')

# Plot the second set of bars, adjusting the x position
bars10 = ax.bar(lengths + 9*bar_width, df['Extreme Wet'], bar_width, label='Extreme Wet', color='#4575b4')

# Plot the second set of bars, adjusting the x position
bars11 = ax.bar(lengths + 10*bar_width, df['Exceptional Wet'], bar_width, label='Exceptional Wet', color='#313695')

# Add labels and title
ax.set_xlabel('States')
ax.set_ylabel('Hectacres')
ax.set_title('July 12th, 2022: Hectacres in Each Drought Category for Top 5 Corn Producing States')
ax.set_xticks(lengths + 5*bar_width)
ax.set_xticklabels(df['Name'])

#Move legend outside of plot
ax.legend(loc='upper right', bbox_to_anchor=(1.45, 1))

#Make sure the legend isn't cut of when saving figure
plt.tight_layout()

#Save figure to temporary files
plt.savefig('demo2.png')

# Show the plot
plt.show()

###Demo 3: Average Vulnerability in each USDM category in Polygon

In [ ]:
#Set Up & Make Request
#group_by Endpoint
endpoint = 'zonal_stats/group_by/polygons'

#Get average vulnerability in each drought category in user defined polygon
#Categories:
params ={
    'group_img_id': 'projects/climate-engine/usdm/weekly/20240116_conus',
    'summary_img_id': 'projects/climate-engine-pro/assets/ce-svi/2020',
    'coordinates': '[[[-126.03,23.89], [-66.45,23.89], [-66.45,49.73], [-126.03,49.73]]]',
    'group_band': 'drought',
    'summary_band': 'b1',
    'group_img_type': 'discrete',
    'area_reducer': 'mean',
    'bins': '[-1,1,2,3,4]'
  }
# Send request to the API
r = requests.get(root_url + endpoint, params=params, headers=headers, verify=True)
response = r.json()
print(response)

In [ ]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(response)
df = pd.concat([df.drop(['group'], axis=1), df['group'].apply(pd.Series)], axis=1)
df

###Demo 4: Population in Temperature Categories over JAS 2023 for California

In [12]:
#Create List of dates to loop through
dates = ['20230701','20230702','20230703','20230704','20230705','20230706','20230707','20230708','20230709','20230710',
         '20230711','20230712','20230713','20230714','20230715','20230716','20230717','20230718','20230719','20230720',
         '20230721','20230722','20230723','20230724','20230725','20230726','20230727','20230728','20230729','20230730',
         '20230731','20230801','20230802','20230803','20230804','20230805','20230806','20230807','20230808','20230809',
         '20230810','20230811','20230812','20230813','20230814','20230815','20230816','20230817','20230818','20230819',
         '20230820','20230821','20230822','20230823','20230824','20230825','20230826','20230827','20230828','20230829',
         '20230830','20230831']

In [15]:
#Define function for API requests
daily_list = []

def get_summaries(date):
  #Set Up & Make Request
  #group_by Endpoint
  endpoint = 'zonal_stats/group_by/custom_asset'

  #Get population in temperature categories in California
  #Categories:
  params ={
    'group_img_id': 'IDAHO_EPSCOR/GRIDMET/' + date,
    'summary_img_id': 'CIESIN/GPWv411/GPW_Population_Count/gpw_v4_population_count_rev11_2020_30_sec',
    'asset_id': 'projects/earthengine-legacy/assets/projects/climate-engine/featureCollections/shp_simplified/ClimateEngine_US_States',
    'sub_choices': 'California',
    'filter_by': 'Name',
    'group_band': 'tmmx',
    'summary_band': 'population_count',
    'group_img_type': 'continuous',
    'area_reducer': 'sum',
    'bins': '[308.15,310.928]'
    }
  # Send request to the API
  r = requests.get(root_url + endpoint, params=params, headers=headers, verify=True)
  response = r.json()

  #Unpack response
  [data] = response

  #Create a data column
  data['Date'] = date

  #Remove Name
  data.pop('Name')

  #Append data to list
  daily_list.append(data)

  #Return List
  return daily_list

In [16]:
#Run function over all dates
for i in dates:
  get_summaries(i)




In [ ]:
#Convert list to pandas dataframe
dt = pd.DataFrame.from_records(daily_list)

# Flatten the 'group' column
dt2 = pd.concat([dt.drop(['group'], axis=1), dt['group'].apply(pd.Series)], axis=1)

#Reformat Date
dt2['Date'] = pd.to_datetime(dt2['Date'], format='%Y%m%d').dt.strftime('%m-%d')

#Rename columns
dt2 = dt2.rename(columns = {'(-inf, 308.15)': 'lt_95', '[308.15, 310.928)':'gte_95_lt100','[310.928, inf)': 'gte_100' })
dt2

In [ ]:
#Plot
# Plot multiple lines
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

#Add Different lines
plt.plot(dt2['Date'], dt2['lt_95'], label='lt_95', color = '#ffeda0', marker='o')
plt.plot(dt2['Date'], dt2['gte_95_lt100'], label='gte_95_lt100', color = '#feb24c', marker='o')
plt.plot(dt2['Date'], dt2['gte_100'], label='gte_100', color = '#f03b20', marker='o')

#Make x-ticks every 5 days
plt.xticks(dt2['Date'][::5])

#Create Labels
plt.xlabel('Date')
plt.ylabel('Population')
plt.title('July 1 - August 31, 2024: Population in different temperature categories')
plt.legend(loc='upper right', bbox_to_anchor=(1.20, 1))
plt.grid(True)

#Make sure the legend isn't cut of when saving figure
plt.tight_layout()

#Save figure to temporary files
plt.savefig('demo4.png')

plt.show()